In [319]:
import os
import numpy as np
import time
from models import ModelParams, ModelShocks, ModelSol
from equations import *
from equations_matrix import calc_X
from solvers import solve_price_and_cost
from functions import generate_rand_params
from optimization import reconstruct_w_hat, objective_w_hat_reduced
from scipy.optimize import minimize
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
sigma = 0.2
N, J = 2, 2

# Generate a shock for each country, shape: (N,)
country_shocks = np.random.normal(loc=0.0, scale=sigma, size=(N,))
# Convert to log-normal shocks and replicate across sectors to get shape (N, J)
lambda_hat_country = np.exp(country_shocks)[:, np.newaxis]  # shape: (N, 1)
lambda_hat_country = np.repeat(lambda_hat_country, J, axis=1)  # shape: (N, J)

# Generate a shock for each sector, shape: (J,)
sector_shocks = np.random.normal(loc=0.0, scale=sigma, size=(J,))
# Convert to log-normal shocks and replicate across countries to get shape (N, J)
lambda_hat_sector = np.exp(sector_shocks)[np.newaxis, :]  # shape: (1, J)
lambda_hat_sector = np.repeat(lambda_hat_sector, N, axis=0)  # shape: (N, J)

# Generate iid shocks for each country-sector pair, shape: (N, J)
lambda_hat_iid = np.exp(np.random.normal(loc=0.0, scale=sigma, size=(N, J)))